In [ ]:
import pandas as pd

import configparser
import os
import glob
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

import hashlib

config = configparser.ConfigParser()
config.read(os.path.join(os.path.expanduser('~'), '.config', 'me.ini'))


engine = create_engine(config['db']['url'])

from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from sqlalchemy import create_engine

import time

In [ ]:
username = config['cgd']['username']
password = config['cgd']['password']

In [ ]:
positions = [
    'Descrição',
    
    'Montante do juro',
    'Montante do imposto',
    'Montante da transferência',
    'Montante original',
    'Montante',
    
    'Data valor',
    'Data do movimento',
    'Tipo de movimento',
    'Saldo contabilístico após movimento',
    'Saldo disponível após movimento',
    'Comentário cliente',
    'Entidade',
    'Número do cartão',
    'N.º Identificação SIBS',
    'Meio de pagamento utilizado',
    'Data início de cálculo do juro',
    'Data fim de cálculo do juro',
    'Conta juros credores',
    'Local',
    'Operação efectuada em',
    'Referência',
    'Valor cativo após movimento',
    'Conta destino',
    'Nº de Transferência',
    'Taxa de câmbio',
    'Contravalor',
    'Descritivo conta destino',
    'Conta destino',
    'Tipo serviço',
    'Finalidade',
    'Nome Ordenante',
    'Banco Ordenante',
    'Swift Ordenante',
    'Número do cheque',
    'Nome do destinatário',
    'Swift do destinatário',
    'Iban do destinatário',
    'Motivo'
]

In [ ]:
movements = pd.read_sql('account_movements', engine)
details = pd.read_sql('account_details', engine)
max_mov_id = movements.movement_id.max() + 1

In [ ]:
display = Display(visible=0, size=(1024, 768))
display.start()

browser = webdriver.Firefox()
browser.set_window_size(1020, 400)
browser.get('https://www.cgd.pt')

time.sleep(3)
# username
elem = browser.find_element_by_id('input_cx1')
elem.send_keys(username + Keys.ENTER)
time.sleep(3)

# Warning OK
elem = browser.find_element_by_id('j_id36')
elem.send_keys(Keys.ENTER)
time.sleep(3)

# Set password
elem = browser.find_element_by_id('passwordInput')
elem.send_keys(password + Keys.ENTER)
time.sleep(3)

# Clear warning
elem = browser.find_element_by_class_name('botao_ok')
elem.send_keys(Keys.ENTER)


elem_menu = browser.find_element_by_id('menuLink101149984')
elem_menu.send_keys(Keys.ENTER)
elem_menu = browser.find_element_by_link_text('Saldos e movimentos')
elem_menu.send_keys(Keys.ENTER)

In [ ]:
mov_id = max_mov_id

for el in range(29):
    
    print('Looking for element: {}'.format(el))
    
    button = browser.find_element_by_id('tdImg{}'.format(el))
    button.click()

    values = browser.find_element_by_class_name('rowdetails').text
    
    button = browser.find_element_by_id('tdImg{}'.format(el))
    button.click()
    
    # ---- convert values to a list of dictionaries
    newvalues = []
    mov_type = None
    for line in values.splitlines():
        for pos in positions:
            # print(str(mov_id) + ' ' + pos + ' - ' + line)
            if pos in line:
                newvalues.append({
                    'type': pos.strip(),
                    'value': line.replace(pos, '').strip(),
                    'movement_id': mov_id
                })
                break
    mov_id += 1
    
    # ---- Create a data frame from the values
    valdf = pd.DataFrame(newvalues)

    global_pos = [
        'Data do movimento',
        'Data valor',
        'Descrição',
        'Montante',
        'Saldo contabilístico após movimento',
        'Saldo disponível após movimento',
        'Tipo de movimento'
    ]
    
    # create a new movements df
    cgdmovements = valdf[valdf['type'].isin(global_pos)].pivot(index='movement_id', columns='type', values='value')

    cgdmovements.columns = [
        'movement_date',
        'value_date',
        'description',
        'ammount',
        'balance',
        'available_balance',
        'movement_type']
    
    # convert a few values
    cgdmovements.value_date = pd.to_datetime(cgdmovements.value_date, format="%d-%m-%Y")

    # add a midnight hour to movements that do not have it
    cgdmovements.ix[cgdmovements.movement_date.str.contains('[0-9]{2}-[0-9]{2}-[0-9]{4}\Z'), 'movement_date'] = \
        cgdmovements.ix[cgdmovements.movement_date.str.contains('[0-9]{2}-[0-9]{2}-[0-9]{4}\Z'), 'movement_date'] + ' 00:00'
    cgdmovements.movement_date = pd.to_datetime(cgdmovements.movement_date, format="%d-%m-%Y %H:%M")

    # convert currency
    cgdmovements.ix[cgdmovements.movement_type == 'Débito', 'ammount'] = \
        '-' + cgdmovements.ix[cgdmovements.movement_type == 'Débito', 'ammount']

    cgdmovements.ammount = cgdmovements.ammount.str.replace(' EUR', '')
    cgdmovements.ammount = cgdmovements.ammount.str.replace('.', '')
    cgdmovements.ammount = cgdmovements.ammount.str.replace(',', '.')
    cgdmovements.ammount = pd.to_numeric(cgdmovements.ammount)


    cgdmovements.balance = cgdmovements.balance.str.replace(' EUR', '')
    cgdmovements.balance = cgdmovements.balance.str.replace('.', '')
    cgdmovements.balance = cgdmovements.balance.str.replace(',', '.')
    cgdmovements.balance = pd.to_numeric(cgdmovements.balance)


    cgdmovements.available_balance = cgdmovements.available_balance.str.replace(' EUR', '')
    cgdmovements.available_balance = cgdmovements.available_balance.str.replace('.', '')
    cgdmovements.available_balance = cgdmovements.available_balance.str.replace(',', '.')
    cgdmovements.available_balance = pd.to_numeric(cgdmovements.available_balance)

    # remove columns not needed, and sort it.
    cgdmovements['account_id'] = 1
    cgdmovements['movement_id'] = cgdmovements.index

    cgdmovements['hash'] = cgdmovements.apply(lambda x:hashlib.md5('{}|{}|{}|{}|{}|{}|{}|{}'.format(
            x['movement_date'], 
            x['value_date'], 
            x['description'], 
            x['ammount'], 
            x['balance'], 
            x['available_balance'], 
            x['movement_type'], 
            x['account_id']).encode('utf-8')).hexdigest(), axis=1)

    mov_hash = cgdmovements.iloc[0]['hash']
    
    
    if len(movements[movements['hash'] == mov_hash]) == 0:
        # no movents. add it to the tables
        print('New movement found.')
        cgdmovements.to_sql('account_movements', engine, if_exists='append', index=False)
        valdf.to_sql('account_details', engine, if_exists='append', index=False)
    else:
        # existing movement. check if there are more details to it
        print('Existing movement. checking details.')
        cur_mov_id = movements[movements['hash'] == mov_hash].iloc[0].movement_id
        curr_details = details[details.movement_id == cur_mov_id]
        
        if len(curr_details) != len(valdf):
            print('New details found. Previous: {}, Now: {}.'.format(len(curr_details), len(valdf)))
            
            engine.execute('delete from account_details where movement_id = {}'.format(cur_mov_id))
            valdf.movement_id = cur_mov_id
            valdf.to_sql('account_details', engine, if_exists='append', index=False)
        

In [ ]:
browser.close()
display.stop()